### Créer les .docx

In [4]:
import os
from win32com import client
import pythoncom

def convert_doc_to_docx(doc_path):
    try:
        pythoncom.CoInitialize()

        word = client.Dispatch("Word.Application")
        doc = word.Documents.Open(doc_path)
        docx_path = os.path.splitext(doc_path)[0] + ".docx"
        doc.SaveAs(docx_path, FileFormat=16)  # 16 correspond au format .docx
        doc.Close()
        word.Quit()
        return docx_path

    except Exception as e:
        print(f"Erreur lors de la conversion de {doc_path}: {e}")
        # Supprime le fichier s'il est corrompu
        try:
            os.remove(doc_path)
            print(f"Fichier corrompu supprimé : {doc_path}")
        except Exception as remove_error:
            print(f"Impossible de supprimer le fichier {doc_path}: {remove_error}")

def convert_docs_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".doc") and not filename.endswith(".docx"):
            doc_path = os.path.join(folder_path, filename)
            docx_path = os.path.splitext(doc_path)[0] + ".docx"

            if not os.path.exists(docx_path):
                print(f"Conversion de {filename} en .docx...")
                convert_doc_to_docx(doc_path)
            else:
                print(f"Le fichier {docx_path} existe déjà. Conversion ignorée.")

# Exemple d'utilisation
if __name__ == "__main__":
    folder_path = r"C:\Users\Colin MININI\Desktop\code\all_syntheses_2"  # Remplacez par le chemin de votre dossier
    convert_docs_in_folder(folder_path)


Conversion de 1-CIR porteur citerne hydrocarbure léger 2011.doc en .docx...
Conversion de 1-CIR PROJET SEMI ALU HYDROCARBURES 2009.doc en .docx...
Conversion de 1-CIR SREM CO2 à auditer 2010.doc en .docx...
Conversion de 2- CIR  porteur GPL 2010.doc en .docx...
Conversion de 3-CIR régulation dosage chlore 2010.doc en .docx...
Conversion de 4- CIR SR projetable 2010_sh.doc en .docx...
Conversion de 7-CIR efficacité transfert d'oxygène 2011.doc en .docx...
Conversion de 8- CIR SR Tout chemin 2011 .doc en .docx...
Conversion de CBIEN_CIR18_Dossier justificatif.doc en .docx...
Conversion de Mallyance_CIR19_VF.doc en .docx...
Conversion de Mallyance_CIR22_Dossier justificatif.doc en .docx...
Conversion de MBT_CIR15_Dossier justificatif_Automate de production&CDE_vRaR.doc en .docx...
Conversion de MBT_CIR15_Dossier justificatif_SPHERE_vRaR.doc en .docx...
Conversion de MBT_CIR18_Dossier justificatif.doc en .docx...
Conversion de MICROBIA_ENVIRONNEMENT_CIR18_Dossier justificatif_vF.doc en .do

### Supprime les .doc

In [5]:
import os

def supprimer_tous_les_docs(dossier):
    for filename in os.listdir(dossier):
        if filename.endswith(".doc") and not filename.endswith(".docx"):
            doc_path = os.path.join(dossier, filename)
            try:
                os.remove(doc_path)
                print(f"Fichier supprimé : {doc_path}")
            except Exception as e:
                print(f"Impossible de supprimer le fichier {doc_path} : {e}")

# Exemple d'utilisation
if __name__ == "__main__":
    dossier = r"C:\Users\Colin MININI\Desktop\code\all_syntheses_2"  # Remplacez par le chemin de votre dossier
    supprimer_tous_les_docs(dossier)


Fichier supprimé : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\1-CIR porteur citerne hydrocarbure léger 2011.doc
Fichier supprimé : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\1-CIR PROJET SEMI ALU HYDROCARBURES 2009.doc
Fichier supprimé : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\1-CIR SREM CO2 à auditer 2010.doc
Fichier supprimé : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\2- CIR  porteur GPL 2010.doc
Fichier supprimé : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\3-CIR régulation dosage chlore 2010.doc
Fichier supprimé : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\4- CIR SR projetable 2010_sh.doc
Fichier supprimé : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\7-CIR efficacité transfert d'oxygène 2011.doc
Fichier supprimé : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\8- CIR SR Tout chemin 2011 .doc
Fichier supprimé : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\CBIEN_CIR18_Dossier justificatif.doc
Fichier supprimé : C:\Us

## Création des fichier état de l'art en txt

### Version fichier

In [3]:

import os
import docx
import openai
from win32com import client

# Fonction pour convertir .doc en .docx
def convert_doc_to_docx(doc_path):
    word = client.Dispatch("Word.Application")
    doc = word.Documents.Open(doc_path)
    docx_path = os.path.splitext(doc_path)[0] + ".docx"
    doc.SaveAs(docx_path, FileFormat=16)  # 16 correspond au format .docx
    doc.Close()
    word.Quit()
    return docx_path

# Fonction pour lire le document Word et extraire son texte
def extract_text_from_word(docx_file):
    doc = docx.Document(docx_file)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Fonction pour utiliser GPT-4 pour détecter la section "État de l'art"
def detect_etat_de_l_art(content, openai_api_key):
    openai.api_key = openai_api_key
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "Vous êtes un assistant qui aide à extraire des sections spécifiques de documents."},
            {"role": "user", "content": f"À partir du texte suivant, extrayez entièrement la partie du texte intitulée 'État de l'art'. Veuillez inclure la bibliographie à la fin de l'état de l'art si elle est présente dans le texte original. Pour repérer si l'état de l'art est terminé, observe si la dernière référence de la bibliographie a été utilisé dans le texte, elle doit apparaitre dans l'état de l'art que tu extrait. Si le texte contient plusieurs 'État de l'art' avec plusiqeurs bibliographies, ne sélectionnez que la première. S'il y a des références bibliographiques dans l'état de l'art (exemple : [4][5]), vous devez les garder. Parfois, l'état de l'art comporte lui-même des sous-sections, vous devez toutes les inclure si c'est le cas. Par exemple, si l'état de l'art commence avec le titre : \"2.2 Etat de l'art\", alors l'état de l'art contient toute la section 2.2 avec ses sous-sections (2.2.1, 2.2.2, ou même 2.2.1.1 par exemple) et elle n'est terminée que lorsque la section 2.3. Pour déterminer la fin de l'état de l'art dans le texte, généralement, l'état de l'art se termine par une bibliographie. Si la section 'État de l'art' est vide ou qu'elle fait moins de 500 caractères ou qu'elle n'existe pas, vous devez retourner : 'Pas d'état de l'art' \n\n{content}"}
        ],
        max_tokens=16384,
        temperature=0.3,
    )
    
    return response.choices[0].message.content

def process_single_word_file(file_path, openai_api_key):
    if not file_path.endswith(".docx"):
        print(f"Le fichier {file_path} n'est pas un fichier Word .docx.")
        return
    
    # Extraire le texte du fichier Word
    document_content = extract_text_from_word(file_path)
    
    # Extraire l'état de l'art
    etat_de_l_art = detect_etat_de_l_art(document_content, openai_api_key)
    
    # Vérifier si le contenu est "Pas d'état de l'art"
    if etat_de_l_art.strip().lower() in ["pas d'état de l'art", "pas d'état de l'art."]:
        print(f"Pas d'état de l'art trouvé dans {os.path.basename(file_path)}.")
    else:
        # Créer le nom du fichier pour l'état de l'art
        output_file_path = os.path.splitext(file_path)[0] + "_etat_de_l_art.txt"
        
        # Écrire l'état de l'art dans un nouveau fichier texte
        with open(output_file_path, "w", encoding="utf-8") as output_file:
            output_file.write(etat_de_l_art)
        
        print(f"L'état de l'art a été sauvegardé dans : {output_file_path}")

# Utilisation du code
if __name__ == "__main__":
    # Chemin vers le fichier Word
    file_path = r"C:\Users\Colin MININI\Desktop\code\all_syntheses_sota\THEGREENBOW_CIR23_Dossier Justificatif_complet.docx"
    
    # Clé API OpenAI (à remplacer par la vôtre)
    openai_api_key = ""
    
    # Traitement d'un seul fichier Word
    process_single_word_file(file_path, openai_api_key)
    

L'état de l'art a été sauvegardé dans : C:\Users\Colin MININI\Desktop\code\all_syntheses_sota\THEGREENBOW_CIR23_Dossier Justificatif_complet_etat_de_l_art.txt


### Version dossier

In [7]:
import os
import docx
import openai
from win32com import client

# Fonction pour convertir .doc en .docx
def convert_doc_to_docx(doc_path):
    word = client.Dispatch("Word.Application")
    doc = word.Documents.Open(doc_path)
    docx_path = os.path.splitext(doc_path)[0] + ".docx"
    doc.SaveAs(docx_path, FileFormat=16)  # 16 correspond au format .docx
    doc.Close()
    word.Quit()
    return docx_path

# Fonction pour lire le document Word et extraire son texte
def extract_text_from_word(docx_file):
    doc = docx.Document(docx_file)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Fonction pour utiliser GPT-4 pour détecter la section "État de l'art"
def detect_etat_de_l_art(content, openai_api_key):
    openai.api_key = openai_api_key
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "Vous êtes un assistant qui aide à extraire des sections spécifiques de documents."},
            {"role": "user", "content": f"À partir du texte suivant, extrayez entièrement la partie du texte intitulée 'État de l'art'. Veuillez inclure la bibliographie à la fin de l'état de l'art si elle est présente dans le texte original. Pour repérer si l'état de l'art est terminé, observe si la dernière référence de la bibliographie a été utilisé dans le texte, elle doit apparaitre dans l'état de l'art que tu extrait. Si le texte contient plusieurs 'État de l'art' avec plusiqeurs bibliographies, ne sélectionnez que la première. S'il y a des références bibliographiques dans l'état de l'art (exemple : [4][5]), vous devez les garder. Parfois, l'état de l'art comporte lui-même des sous-sections, vous devez toutes les inclure si c'est le cas. Par exemple, si l'état de l'art commence avec le titre : \"2.2 Etat de l'art\", alors l'état de l'art contient toute la section 2.2 avec ses sous-sections (2.2.1, 2.2.2, ou même 2.2.1.1 par exemple) et elle n'est terminée que lorsque la section 2.3. Pour déterminer la fin de l'état de l'art dans le texte, généralement, l'état de l'art se termine par une bibliographie. Si la section 'État de l'art' est vide ou qu'elle fait moins de 500 caractères ou qu'elle n'existe pas, vous devez retourner : 'Pas d'état de l'art' \n\n{content}"}
        ],
        max_tokens=16384,
        temperature=0.3,
    )
    
    return response.choices[0].message.content

def process_word_files_in_folder(folder_path, openai_api_key):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if filename.endswith(".docx"):
            # Extraire le texte du fichier Word
            document_content = extract_text_from_word(file_path)
            
            # Extraire l'état de l'art
            etat_de_l_art = detect_etat_de_l_art(document_content, openai_api_key)
            
            # Vérifier si le contenu est "Pas d'état de l'art"
            if etat_de_l_art.strip().lower() == "pas d'état de l'art." or etat_de_l_art.strip().lower() == "pas d'état de l'art":
                print(f"Pas d'état de l'art trouvé dans {filename}. Aucun fichier texte ne sera créé.")
            else:
                # Créer le fichier texte avec le même nom que le fichier Word
                output_filename = os.path.splitext(filename)[0] + ".txt"
                output_path = os.path.join(folder_path, output_filename)
                
                # Écrire le contenu dans le fichier texte
                with open(output_path, "w", encoding="utf-8") as text_file:
                    text_file.write(etat_de_l_art)
                
                print(f"Fichier '{output_filename}' créé avec l'extrait de l'état de l'art.")

# Utilisation du code
if __name__ == "__main__":
    # Chemin vers le dossier contenant les fichiers Word
    folder_path = r"C:\Users\Colin MININI\Desktop\code\all_syntheses_2"
    
    # Clé API OpenAI (à remplacer par la vôtre)
    openai_api_key = ""
    
    # Traitement des fichiers Word dans le dossier
    process_word_files_in_folder(folder_path, openai_api_key)


Fichier '1-CIR porteur citerne hydrocarbure léger 2011.txt' créé avec l'extrait de l'état de l'art.
Fichier '1-CIR PROJET SEMI ALU HYDROCARBURES 2009.txt' créé avec l'extrait de l'état de l'art.
Fichier '1-CIR SREM CO2 à auditer 2010.txt' créé avec l'extrait de l'état de l'art.
Fichier '2- CIR  porteur GPL 2010.txt' créé avec l'extrait de l'état de l'art.
Fichier '3-CIR régulation dosage chlore 2010.txt' créé avec l'extrait de l'état de l'art.
Fichier '4- CIR SR projetable 2010_sh.txt' créé avec l'extrait de l'état de l'art.
Fichier '7-CIR efficacité transfert d'oxygène 2011.txt' créé avec l'extrait de l'état de l'art.
Fichier '8- CIR SR Tout chemin 2011 .txt' créé avec l'extrait de l'état de l'art.
Fichier 'CBIEN_CIR16_Dossier_Justificatif_VF.txt' créé avec l'extrait de l'état de l'art.
Fichier 'CBIEN_CIR18_Dossier justificatif.txt' créé avec l'extrait de l'état de l'art.
Pas d'état de l'art trouvé dans DMG_CIR17_Description du projet.docx. Aucun fichier texte ne sera créé.
Fichier 'M

## Créer les fichiers bullet points

### Version fichier

In [5]:

import os
import openai

def detect_etat_de_l_art(content, openai_api_key):
    openai.api_key = openai_api_key
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "Tu es un assistant spécialisé dans la simplification et la synthèse de textes techniques complexes. Ton rôle est de convertir un texte qui est l'état de l'art pour des dossiers de Crédit Impôt Recherche (CIR) en bullet points concis et précis. Ces bullet points doivent contenir toutes les informations présentes dans le texte en entrée. Ta réponse doit être organisée, couvrir tous les aspects du texte original. Un bullet point sur les limites si elles sont mentionnées dans le texte. Ne simplifie pas, mais veille à ce que chaque point soit clair. Si une information du texte d'entrée contient une référence bibliographique (exemple : '[4]'), le bullet point correspondant doit contenir la même référence. A la fin tu doit réecrire la bibliographie qui se situe à la fin de l'état de l'art "},
            {"role": "user", "content": f"Voici le texte :\n\n{content}"}
        ],
        max_tokens=6384,
        temperature=0.3,
    )
    
    return response.choices[0].message.content

def process_single_text_file(file_path, openai_api_key):
    if not file_path.endswith(".txt"):
        print("Le fichier sélectionné n'est pas un fichier texte.")
        return
    
    # Lire le contenu du fichier texte
    with open(file_path, "r", encoding="utf-8") as file:
        etat_de_l_art_content = file.read()
    
    # Afficher l'état de l'art original
    print("État de l'art original :")
    print(etat_de_l_art_content)
    print("\nGénération des bullet points...\n")
    
    # Générer les bullet points à partir de l'état de l'art
    bullet_points = detect_etat_de_l_art(etat_de_l_art_content, openai_api_key)
    
    # Afficher les bullet points générés
    print("Bullet points générés :")
    print(bullet_points)
    
    # Créer le nom du fichier pour les bullet points
    output_file_path = os.path.splitext(file_path)[0] + "_bullets.txt"
    
    # Écrire les bullet points dans un nouveau fichier
    with open(output_file_path, "w", encoding="utf-8") as output_file:
        output_file.write(bullet_points)
    
    print(f"Les bullet points ont été sauvegardés dans : {output_file_path}")

if __name__ == "__main__":
    # Chemin vers le fichier texte
    file_path = r"C:\Users\Colin MININI\Desktop\code\all_syntheses_sota\Consensus_Math.txt"
    
    # Clé API OpenAI (à remplacer par la vôtre)
    openai_api_key = ""
    
    # Traitement du fichier texte spécifié
    process_single_text_file(file_path, openai_api_key)


État de l'art original :
**État de l’art**

Zero Trust Network Access (ZTNA)  
L’article [1] introduit le ZTN (Zero Trust Network) en soulignant l’évolution de la technologie informatique, l’utilisation du cloud, de l’edge computing et de l’arrivée massive de l’IIoT (Industrial Internet of Things). Parallèlement à cela, le nombre d’attaques informatiques explose et celles-ci sont de plus en plus sophistiquées. Ainsi, les systèmes d’AC (Access Control) doivent s’améliorer drastiquement, s’adaptant de façon dynamique pour prendre en compte ces nouveaux facteurs de risques. Ces systèmes devant inclure de nombreux facteurs tels que le degré de confiance en l'utilisateur et le dispositif, le contexte de l'utilisateur et du dispositif, c'est-à-dire l'emplacement, l'heure du jour, le type de tâche ainsi que le niveau de menace actuel pour la sécurité dans l'environnement immédiat de l'utilisateur [2]. En outre, le niveau d'accès attribué à un dispositif ou à un utilisateur peut changer au fil

### Version dossier

In [13]:
import os
import openai

def detect_etat_de_l_art(content, openai_api_key):
    openai.api_key = openai_api_key
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "Tu es un assistant spécialisé dans la simplification et la synthèse de textes techniques complexes. Ton rôle est de convertir un texte qui est l'état de l'art pour des dossiers de Crédit Impôt Recherche (CIR) en bullet points concis et précis. Ces bullet points doivent contenir toutes les informations présentes dans le texte en entrée. Ta réponse doit être organisée, couvrir tous les aspects du texte original. Un bullet point sur les limites si elles sont mentionnées dans le texte. Ne simplifie pas, mais veille à ce que chaque point soit clair. Si une information du texte d'entrée contient une référence bibliographique (exemple : '[4]'), le bullet point correspondant doit contenir la même référence. A la fin tu doit réecrire la bibliographie qui se situe à la fin de l'état de l'art "},
            {"role": "user", "content": f"Voici le texte :\n\n{content}"}
        ],
        max_tokens=6384,
        temperature=0.3,
    )
    
    return response.choices[0].message.content

def process_text_files_in_folder(folder_path, openai_api_key):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") and not filename.endswith("_bullets.txt"):
            file_path = os.path.join(folder_path, filename)
            
            # Lire le contenu du fichier texte
            with open(file_path, "r", encoding="utf-8") as file:
                etat_de_l_art_content = file.read()
            
            # Afficher l'état de l'art original
            print(f"État de l'art original pour {filename} :")
            #print(etat_de_l_art_content)
            print("\nGénération des bullet points...\n")
            
            # Générer les bullet points à partir de l'état de l'art
            bullet_points = detect_etat_de_l_art(etat_de_l_art_content, openai_api_key)
            
            # Afficher les bullet points générés
            print(f"Bullet points générés pour {filename} :")
            #print(bullet_points)
            
            # Créer le nom du fichier pour les bullet points
            output_file_path = os.path.splitext(file_path)[0] + "_bullets.txt"
            
            # Écrire les bullet points dans un nouveau fichier
            with open(output_file_path, "w", encoding="utf-8") as output_file:
                output_file.write(bullet_points)
            
            print(f"Les bullet points ont été sauvegardés dans : {output_file_path}\n")

if __name__ == "__main__":
    # Chemin vers le dossier contenant les fichiers texte
    folder_path = r"C:\Users\Colin MININI\Desktop\code\all_syntheses_2"
    
    # Clé API OpenAI (à remplacer par la vôtre)
    openai_api_key = ""
    
    # Traitement des fichiers texte dans le dossier
    process_text_files_in_folder(folder_path, openai_api_key)


État de l'art original pour 1-CIR porteur citerne hydrocarbure léger 2011.txt :

Génération des bullet points...

Bullet points générés pour 1-CIR porteur citerne hydrocarbure léger 2011.txt :
Les bullet points ont été sauvegardés dans : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\1-CIR porteur citerne hydrocarbure léger 2011_bullets.txt

État de l'art original pour 2- CIR  porteur GPL 2010.txt :

Génération des bullet points...

Bullet points générés pour 2- CIR  porteur GPL 2010.txt :
Les bullet points ont été sauvegardés dans : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\2- CIR  porteur GPL 2010_bullets.txt

État de l'art original pour 3-CIR régulation dosage chlore 2010.txt :

Génération des bullet points...

Bullet points générés pour 3-CIR régulation dosage chlore 2010.txt :
Les bullet points ont été sauvegardés dans : C:\Users\Colin MININI\Desktop\code\all_syntheses_2\3-CIR régulation dosage chlore 2010_bullets.txt

État de l'art original pour 4- CIR SR projetable 

## Ajoute les lignes du dataset

### Version fichier

In [ ]:
import os
import json

def lire_fichier_txt(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read().strip()

def generer_json_dataset(texte_file_path, bullets_file_path):
    # Lire le contenu du texte principal
    texte_content = lire_fichier_txt(texte_file_path)
    
    # Lire le contenu des bullet points
    bullets_content = lire_fichier_txt(bullets_file_path)
    
    # Contenu du champ "system"
    system_content = (
        "Tu es un assistant spécialisé dans la rédaction 'État de l'art' pour des dossiers de Crédit Impôt Recherche (CIR). "
        "Ton objectif est de rédiger un état de l'art complet à partir des bullet points que tu reçois en input. Le texte doit être clair, "
        "précis et bien structuré pour s'inscrire dans une synthèse technique du projet de l'entreprise. Tu dois utiliser un langage technique, "
        "respecter la rigueur scientifique, et t'assurer que la structure soit logique et cohérente. Tu dois inclure des détails techniques pertinents, "
        "des comparaisons, des références aux innovations, et souligner les défis techniques et/ou les lacunes dans la littérature existante. "
        "Rédige le texte de manière fluide, en introduisant les concepts progressivement et en les expliquant de façon détaillée tout en restant concis et professionnel."
    )
    
    # Créer le format JSON pour le dataset
    dataset_entry = {
        "messages": [
            {"role": "system", "content": system_content},
            {"role": "user", "content": bullets_content},
            {"role": "assistant", "content": texte_content}
        ]
    }
    
    return dataset_entry

def ajouter_au_fichier_jsonl(dataset_entry, output_file_path):
    # Ajouter l'entrée JSON au fichier JSONL
    with open(output_file_path, "a", encoding="utf-8") as output_file:
        json.dump(dataset_entry, output_file, ensure_ascii=False)
        output_file.write("\n")  # Ajouter une nouvelle ligne pour respecter le format JSONL

if __name__ == "__main__":
    # Chemin vers les fichiers texte
    texte_file_path = r"C:\Users\Colin MININI\Desktop\code\all_synthese\Lallemand_CIR15.txt"
    bullets_file_path = r"C:\Users\Colin MININI\Desktop\code\all_synthese\Lallemand_CIR15_bullets.txt"
    
    # Générer le JSON pour le dataset
    dataset_entry = generer_json_dataset(texte_file_path, bullets_file_path)
    
    # Chemin pour ajouter l'entrée au fichier JSONL
    output_file_path = r"C:\Users\Colin MININI\Desktop\code\FT_Github\dataset.jsonl"
    
    # Ajouter l'entrée au fichier JSONL
    ajouter_au_fichier_jsonl(dataset_entry, output_file_path)
    
    print(f"L'entrée a été ajoutée au fichier JSONL sous : {output_file_path}")


L'entrée a été ajoutée au fichier JSONL sous : C:\Users\Colin MININI\Desktop\code\FT_Github\dataset.jsonl


### Version dossier

In [14]:
import os
import json

def lire_fichier_txt(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read().strip()

def generer_json_dataset(texte_file_path, bullets_file_path):
    # Lire le contenu du texte principal
    texte_content = lire_fichier_txt(texte_file_path)
    
    # Lire le contenu des bullet points
    bullets_content = lire_fichier_txt(bullets_file_path)
    
    # Contenu du champ "system"
    system_content = (
        "Tu es un assistant spécialisé dans la rédaction 'État de l'art' pour des dossiers de Crédit Impôt Recherche (CIR). "
        "Ton objectif est de rédiger un état de l'art complet à partir des bullet points que tu reçois en input. Le texte doit être clair, "
        "précis et bien structuré pour s'inscrire dans une synthèse technique du projet de l'entreprise. Tu dois utiliser un langage technique, "
        "respecter la rigueur scientifique, et t'assurer que la structure soit logique et cohérente. Tu dois inclure des détails techniques pertinents, "
        "des comparaisons, des références aux innovations, et souligner les défis techniques et/ou les lacunes dans la littérature existante. "
        "Rédige le texte de manière fluide, en introduisant les concepts progressivement et en les expliquant de façon détaillée tout en restant concis et professionnel."
    )
    
    # Créer le format JSON pour le dataset
    dataset_entry = {
        "messages": [
            {"role": "system", "content": system_content},
            {"role": "user", "content": bullets_content},
            {"role": "assistant", "content": texte_content}
        ]
    }
    
    return dataset_entry

def ajouter_au_fichier_jsonl(dataset_entry, output_file_path):
    # Ajouter l'entrée JSON au fichier JSONL
    with open(output_file_path, "a", encoding="utf-8") as output_file:
        json.dump(dataset_entry, output_file, ensure_ascii=False)
        output_file.write("\n")  # Ajouter une nouvelle ligne pour respecter le format JSONL

def process_folder(folder_path, output_file_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt") and not filename.endswith("_bullets.txt"):
            texte_file_path = os.path.join(folder_path, filename)
            bullets_file_path = os.path.splitext(texte_file_path)[0] + "_bullets.txt"
            
            # Vérifie si le fichier des bullet points existe
            if os.path.exists(bullets_file_path):
                # Générer le JSON pour le dataset
                dataset_entry = generer_json_dataset(texte_file_path, bullets_file_path)
                
                # Ajouter l'entrée au fichier JSONL
                ajouter_au_fichier_jsonl(dataset_entry, output_file_path)
                
                print(f"L'entrée pour {filename} a été ajoutée au fichier JSONL.")

if __name__ == "__main__":
    # Chemin vers le dossier contenant les fichiers texte
    folder_path = r"C:\Users\Colin MININI\Desktop\code\all_synthese"
    
    # Chemin pour ajouter les entrées au fichier JSONL
    output_file_path = r"C:\Users\Colin MININI\Desktop\code\FT_Github\dataset_sota_4omini_full.jsonl"
    
    # Traitement des fichiers texte dans le dossier
    process_folder(folder_path, output_file_path)
    
    print(f"Traitement terminé. Les entrées ont été ajoutées au fichier JSONL sous : {output_file_path}")


L'entrée pour A2L_CIR16_Dossier justificatif.txt a été ajoutée au fichier JSONL.
L'entrée pour A2L_CIR17_Dossier justificatif.txt a été ajoutée au fichier JSONL.
L'entrée pour A2L_CIR18_Dossier justificatif.txt a été ajoutée au fichier JSONL.
L'entrée pour A2L_CIR19_Dossier Justificatif_VF 1.txt a été ajoutée au fichier JSONL.
L'entrée pour A2L_CIRCII2020_2021 Dossier justificatif_VF.txt a été ajoutée au fichier JSONL.
L'entrée pour AGIL IT_CIR19_Dossier justificatif.txt a été ajoutée au fichier JSONL.
L'entrée pour AgilIT_CIR18_Dossier justificatif.txt a été ajoutée au fichier JSONL.
L'entrée pour Akeneo_CIR15_Dossier justificatif_PIM_Tooling_vRaR.txt a été ajoutée au fichier JSONL.
L'entrée pour AKENEO_CIR16_Dossier justificatif.txt a été ajoutée au fichier JSONL.
L'entrée pour Akeneo_CIR17_Dossier justificatif_vf.txt a été ajoutée au fichier JSONL.
L'entrée pour Akeneo_CIR17_Dossier justificatif_vf_etat_de_l_art.txt a été ajoutée au fichier JSONL.
L'entrée pour AKENEO_CIR19_Dossier 